In [1]:
from utils.models_dot import GMF, MLP, NeuFM
from utils.dataset import RatingsDataset
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

import math

import os

import pandas as pd

import numpy as np

In [23]:
users = 3974
movies = 3564

train_dataloader = DataLoader(
    RatingsDataset(
#         "train_data/train.csv",
        "train_data/total.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=1024,
    num_workers=8,
    shuffle=True,
)
val_dataloader = None
# val_dataloader = DataLoader(
#     RatingsDataset(
#         "train_data/validation.csv",
#         "user_id",
#         "movie_id",
#         "rating",
#     ),
#     batch_size=1024,
#     num_workers=4,
#     shuffle=True,
# )

In [4]:
def train(model, lr, epochs, train_dataloader, val_dataloader=None, show_loss=True):
    criterion = nn.MSELoss().cuda()
    optimizer = opt.Adam(model.parameters(), lr=lr)
    
    avg_losses = []
    
    model_state = None
    best_epoch = 0
    prev_val_loss = math.inf
    for epoch in tqdm(range(epochs)):
        n_batches = len(train_dataloader)
        avg_loss = 0
        val_loss = 0

        # Train step
        for i_batch, (vus, vis, rs) in enumerate(train_dataloader):
            vus = vus.cuda()
            vis = vis.cuda()
            rs = rs.cuda()

            optimizer.zero_grad()
            y_hat = model(vus, vis)

            loss = criterion(y_hat, rs)
            loss.backward()
            optimizer.step()

            avg_loss += math.sqrt(float(loss.detach().cpu()))
        
        avg_loss /= n_batches
        
        # Val step
        if val_dataloader is not None:
            with torch.no_grad():
                for val_vus, val_vis, val_rs in val_dataloader:
                    val_vus = val_vus.cuda()
                    val_vis = val_vis.cuda()
                    val_rs = val_rs.cuda()

                    val_pred = model(val_vus, val_vis)
                    val_loss += math.sqrt(float(criterion(val_pred, val_rs).detach().cpu()))

            val_loss /= len(val_dataloader)
            if show_loss:
                print(f"epoch: {epoch+1}, train_loss: {avg_loss}, val_loss: {val_loss}")
            
            avg_losses.append([avg_loss, val_loss])
        else:
            val_loss = avg_loss
            if show_loss:
                print(f"epoch: {epoch+1}, train_loss: {avg_loss}")
            
            avg_losses.append([avg_loss])
                  
        if val_loss < prev_val_loss:
            prev_val_loss = val_loss
            model_state = model.state_dict()
            best_epoch = epoch
        
    return best_epoch, prev_val_loss, model_state, avg_losses

In [25]:
gmf = GMF(users, movies, 32).cuda()
mlp = MLP(users, movies, 16, [16, 8]).cuda()

In [6]:
import torch
import torch.nn as nn
import torch.optim as opt

In [26]:
gmf_best_epoch, gmf_val_loss, gmf_model_state, gmf_losses = train(
    gmf, lr=0.002, epochs=20, train_dataloader=train_dataloader, val_dataloader=val_dataloader, show_loss=True
)

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 1, train_loss: 2.08454979032049
epoch: 2, train_loss: 0.9564041470523625
epoch: 3, train_loss: 0.9235671382830273
epoch: 4, train_loss: 0.9177757481182734
epoch: 5, train_loss: 0.9140887897370784
epoch: 6, train_loss: 0.9087526553144059
epoch: 7, train_loss: 0.8980217591817914
epoch: 8, train_loss: 0.8837223432396143
epoch: 9, train_loss: 0.86897882040214
epoch: 10, train_loss: 0.8537239440066102
epoch: 11, train_loss: 0.8375871614823431
epoch: 12, train_loss: 0.8203816857719832
epoch: 13, train_loss: 0.8028675905023686
epoch: 14, train_loss: 0.7850624420818767
epoch: 15, train_loss: 0.7670186496042098
epoch: 16, train_loss: 0.7497651685687213
epoch: 17, train_loss: 0.7330886256665203
epoch: 18, train_loss: 0.7178500240167585
epoch: 19, train_loss: 0.7044686225973202
epoch: 20, train_loss: 0.6924959869139281


In [27]:
mlp_best_epoch, mlp_val_loss, mlp_model_state, mlp_losses = train(
    mlp, lr=0.01, epochs=20, train_dataloader=train_dataloader, val_dataloader=val_dataloader, show_loss=True
)

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 1, train_loss: 1.113686145088827
epoch: 2, train_loss: 0.9124033741900665
epoch: 3, train_loss: 0.907103833743733
epoch: 4, train_loss: 0.8971309450226933
epoch: 5, train_loss: 0.8870410838186794
epoch: 6, train_loss: 0.8814227042221919
epoch: 7, train_loss: 0.8765019341196565
epoch: 8, train_loss: 0.873032011891426
epoch: 9, train_loss: 0.869530068736749
epoch: 10, train_loss: 0.8663802732521686
epoch: 11, train_loss: 0.8620296689833017
epoch: 12, train_loss: 0.85774557222161
epoch: 13, train_loss: 0.8543326394962165
epoch: 14, train_loss: 0.8516038832668664
epoch: 15, train_loss: 0.8488378400886802
epoch: 16, train_loss: 0.8462113710768087
epoch: 17, train_loss: 0.8435848022567924
epoch: 18, train_loss: 0.8407049332403111
epoch: 19, train_loss: 0.8383363143832482
epoch: 20, train_loss: 0.8350623811849085


In [28]:
gmf.load_state_dict(gmf_model_state)
mlp.load_state_dict(mlp_model_state)

neufm = NeuFM(
    gmf,
    mlp,
    alpha=0.5,
).cuda()

In [29]:
neufm_best_epoch, neufm_val_loss, neufm_model_state, neufm_losses = train(
    neufm, lr=0.001, epochs=3, train_dataloader=train_dataloader, val_dataloader=val_dataloader, show_loss=True
)

  0%|          | 0/3 [00:00<?, ?it/s]

epoch: 1, train_loss: 0.7079073596177952
epoch: 2, train_loss: 0.6927137859793724
epoch: 3, train_loss: 0.6806991213498536


## Predict

In [30]:
neufm.eval()
    
test_dataloader = DataLoader(
    RatingsDataset(
        "train_data/test.csv",
        "user_id",
        "movie_id",
    ), 
    batch_size=1024,
    num_workers=12
)

test_predictions = []

for vus, vis in test_dataloader:
    vus = vus.cuda()
    vis = vis.cuda()

    pred = torch.clip(neufm(vus, vis), 1, 5).cpu().ravel().tolist()
    test_predictions += pred

test_csv = pd.read_csv("../../data/test_data.csv")

out_df = pd.DataFrame.from_dict(
    {
        "id": list(test_csv["id"]),
        "rating": test_predictions
    }
)

out_df.to_csv(f"/home/nubol23/Desktop/Codes/USP/SCC5966/kaggle/notebooks/Project-NeuMF/outputs_csv/neumf_dot/neumf_dot_20.csv", index=False)
out_df.head()

,id,rating
0,0,3.158576
1,1,3.197291
2,2,2.687427
3,3,3.743812
4,4,3.104539
